In [1]:
# Data
import pandas as pd
import numpy as np
import country_converter as coco

In [2]:
df = pd.read_csv('Data/data_science_jobs_dataset.csv')

In [3]:
df.sample(3)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
78,2022,EN,FT,Data Analyst,50000,USD,50000,AR,100,AR,L
354,2022,SE,FT,Data Engineer,156600,USD,156600,US,100,US,M
265,2022,SE,FT,Data Scientist,260000,USD,260000,US,100,US,M


## Cleaning

### Abreviations

We'll change the abreviations to the actual meaning for extra clarity:
- Experience level:
    - EN: Junior
    - MI: Intermediate
    - SE: Senior
    - EX: Executive
- Employment:
    - FT: Full time
    - PT: Part time
    - CT: Contract
    - FL: Freelance
- Country:
    - Every ISO2 abreviation is changed to the short name of the country
- Remote:
    - 100: Fully Remote
    - 50: Partially Remote
    - 0: No remote work
- Company size:
    - S: Small (less than 50 employees)
    - M: Medium (between 50 and 250 employees)
    - L: Large (more than 250 employees)

In [4]:
experience_rename = {"EN":"Junior", "MI":"Intermediate", "SE":"Senior", "EX":"Executive"}
employment_rename = {"FT":"Full time", "PT":"Part time", "CT":"Contract", "FL":"Freelance"}
country_rename = coco.CountryConverter().get_correspondence_dict('ISO2', 'name_short')
country_rename["GB"] = "United Kingdom" #GB'S conversion is missing from the premade dictionary, so we add it manually
remote_rename = {100:"Fully remote", 50:"Partially remote", 0:"No remote work"}
size_rename = {"S":"Small", "M":"Medium", "L":"Large"}

In [5]:
df = df.replace({"experience_level": experience_rename, "employment_type":employment_rename, 
                 "employee_residence":country_rename, "remote_ratio":remote_rename, 
                 "company_location":country_rename, "company_size":size_rename})
df.sample(3)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
589,2021,Intermediate,Full time,Applied Data Scientist,68000,CAD,54238,United Kingdom,Partially remote,Canada,Large
396,2022,Intermediate,Full time,Data Scientist,102100,USD,102100,United States,No remote work,United States,Medium
503,2022,Intermediate,Full time,Data Scientist,150000,PLN,34650,Poland,Fully remote,Poland,Large


### Droping columns

To be able to make comparisons, we are only interested in the salary in USD. We will drop the columns salary and salary_currency.

In [6]:
df = df.drop(["salary", "salary_currency"], axis=1)
df.sample(3)

,work_year,experience_level,employment_type,job_title,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
294,2022,Senior,Full time,Data Scientist,120160,United States,Fully remote,United States,Medium
336,2022,Senior,Full time,Data Architect,90700,Canada,Fully remote,Canada,Medium
148,2022,Senior,Full time,Data Engineer,130000,United States,Fully remote,United States,Medium


In [7]:
df.to_csv("data/cleaned_data_science_jobs_dataset.csv", index=False)

# EDA

## Feature analysis univariate

### Work Year

<div>
<img src="Images/Work_year.png" width="400"/>
</div>

Every year there is a noticable increase in the number of data in our dataset. More available jobs related to data worldwide could explain a small difference, but the reason for an increase this big is that the data is entered voluntarily and the site that collects it has been gaining popularity over the last 2 years.

### Experience level

<div>
<img src="Images/Experience_level.png" width="700"/>
</div>

More than half of the positions are Senior, and aproximately one out of three are intermediate, one out of seven are junior and only one out of 25 are executive level positions

### Employment type

<div>
<img src="Images/Employment_type.png" width="400"/>
</div>

Almost every single job listed is a full time job.

### Job title

In [8]:
len(df["job_title"].unique())

52

<div>
<img src="Images/Job_title.png" width="900"/>
</div>

There are 52 unique job titles. Some of them are very niche, so we will focus on the 10 most common. The vast majority of data related employees are either Data Engineers, Data Scientists, Data Analysts or Machine Learning Engineers; over 80% of all job titles fall in these 4 categories.

### Salary (USD)

<div>
<img src="Images/Salary.png" width="500"/>
</div>

<div>
<img src="Images/Salary_2.png" width="1000"/>
</div>

The first graph shows that there is a large number of outliers in our dataset. The maximum salary is 900k\\$ per year! 

The second graph is the same boxplot as the first with the outliers filtered out, to make it easier to understand. There we can see that half of all the salaries are between 57k\\$ and 143k\\$ (first and third quartiles), with the median being at around 95k\\$.

The third graph, a bar plot of binned salaries (each bin having a range of 30k\\$) shows that the salaries follow a distribution close to normal, with some outliers and with an obvious lower bound of 0\\$. The mean of this distribution whould be in the 90k-120k\\$ bin, which will be checked in the next block of code:

In [9]:
mean = round(df["salary_in_usd"].mean(),2)
print(f"The mean of all salaries is {mean}$")

The mean of all salaries is 118856.37$


As expected, the mean falls in that bin. Lets check its value if we drop the outliers:

In [10]:
mean = round(df[(df["salary_in_usd"] < df["salary_in_usd"].quantile(0.75))]["salary_in_usd"].mean(),2)
print(f"The mean of salaries (excluding outliers) is {mean}$")

The mean of salaries (excluding outliers) is 86997.91$


### Employee residence

In [11]:
len(df["employee_residence"].unique())

62

<div>
<img src="Images/Employee_residence.png" width="900"/>
</div>

As with job titles, there are too many employee countries of residence to show in a graph (62), so only the 10 most common ones were plotted. The vast majority of our data (68.5%) comes from the United States.

### Remote Ratio

<div>
<img src="Images/Remote_work.png" width="500"/>
</div>

Interestingly, more than half of all the jobs are fully remote. Later we will see the evolution over time of this percentage, to see wether covid has had any impact on it.

### Company Location

<div>
<img src="Images/Company_location.png" width="900"/>
</div>

Similarly to employee residence, the country with the greater amount of jobs is by far the United States, with over 70% of all the share. The difference in percentage between company location and employee residence (1.5% in the case of USA) is explained by people working remotely from other countries. This phenomenon will be explored in more detail later.

### Company Size

<div>
<img src="Images/Company_size.png" width="700"/>
</div>

Medium companies (companies with more than 50 employees but less than 250 employees) make up more than half of all companies in the data.

## Feature analysis multivariate

### Experience level by job title

<div>
<img src="Images/Experience_level_by_job_title.png" width="900"/>
</div>

This bar plot is the same as the one shown before in the job titles section, but this time only the 8 most common job titles are shown, and each job is split by experience level (junior, mid, senior and executive). Inside each job title the percentage of each experience level (regarding that specific job title) is shown as text. 

The jobs with the biggest percentage of juniors are Research Scientists, Machine Learning Engineers, Data Scientists and Data Analysts.

The biggest percentage of intermediate level employees are in Research Science, Data Science and Data Engineering jobs

Seniors are the biggest percentage in Data Architecture and Data Science Managing jobs.

Executives occupy the biggest percentage as Analytics Engineers and Data Science Managers.

### Company size by job title

<div>
<img src="Images/Company_size_by_job_title.png" width="1000"">
</div>

Without taking job title into account, company size percentages are 30% large, 58% medium and 12% small. Taking this into account, the most noticeable insights that follow from this graph are:

- Small companies don't seem to need Data Engineers, Data Managers or Data Architects.
- Medium companies have a large percentage of Data Analysts, Data Architects and Analytics Engineers.
- Large companies require an unusualy large amount of Research Scientists (compared to small and medium companies).

### Remote ratio by work year

<div>
<img src="Images/Remote_ratio_by_work_year.png" width="900"/>
</div>

<div>
<img src="Images/Remote_ratio_by_work_year_2.png" width="800" style="float: left; margin-right: 150px;"/>
</div>

During the last 2 years the percentage of people working either 100% remote or doing no remote work at all has increased considerably at the expense of people working partially remote. This could be attributed to a large number of companies trying some degree of remote work for their employees for the first time during the most severe stage of the covid pandemic (2020-2021). Then by 2022 each company already decided if they wanted to focus on remote work or not.

### Remote Ratio by experience level

<div>
<img src="Images/Remote_ratio_by_experience_level.png" width="800"">
</div>

There is no significant difference in remote ratio for each experience level.

### Remote Ratio by company size

<div>
<img src="Images/Remote_ratio_by_company_size.png" width="1000"">
</div>

There is no significant difference to remote ratio for each company size.

### Of all employees working abroad, how many are in each country (in % of total)

<div>
<img src="Images/Percent_of_abroad.png" width="900"">
</div>

Not to be confused with percentage of people working from abroad for each country, this graph shows how many of all the people working from abroad are in each country. Almost half of the people working abroad work in the USA, around 10% in the UK and around 10% in germany. Therefore, someone looking to work abroad should start applying for jobs in those countries.

### Experience level by company size

<div>
<img src="Images/Experience_level_by_company_size.png" width="900"">
</div>

Small companies have the largest percentage of junior employees, while medium companies havealmost no junior employees at all. For someone wanting to start a career in data it would be advisable to focus on applying to either small or large companies.

### Salary by work year

<div>
<img src="Images/Salary_by_work_year.png" width="450"">
</div>

Last year (2022) there has been a significant improvement on salaries, with 50% of the salaries being in the range of around 80k\\$ to 160k\\$, compared to 2021's interquartile range of around 50k\\$ to 130k\\$.

### Salary by experience level

<div>
<img src="Images/Salary_by_experience_level.png" width="1300"">
</div>

All salary by experience level histograms are shaped like normal distributions (except for junior, which is clearly affected by the lower bound of 0). The mean is clearly increasing with exerience level, but the difference between senior and executive level salaries doesn't appear to be that much. This is partly due to dropping the outliers prior to drawing the plot: most of these outliers are people working as executives.

### Salary by job title

<div>
<img src="Images/Salary_by_job_title.png" width="1300"">
</div>

Jobs in this list can be separated into 3 groups:

- Least paying jobs: Research scientists are by far the least paying job. Their interquartile range is the lowest and it's width also is (meaning there is not much variation within those salaries). Its median is about 60k\\$.
- Mid paying jobs: Data Engineers, Data Scientists and Data Analysts are not only the most common titles in data related jobs but their salaries are also the most average. There is a small difference between them, with Engineers earning more than Scientists, and them earning more than Analysts.
- Top paying jobs: Data Science Manager, Data Architect and Analytics Engineer are managing positions. As we've seen before, most of them are seniors or executives and as such have larger payrolls.

### Salary by company size

<div>
<img src="Images/Salary_by_company_size.png" width="500"">
</div>

There is not a big difference in salaries between large and medium companies, but small companies have significantly lower salaries

### Salary by company location

<div>
<img src="Images/Salary_by_company_location.png" width="1300"">
</div>

Only countries with a significant amount of data were plotted. Overall, north american and japanese companies are the ones that pay the most, followed by west european ones. Inside west Europe, United Kingdom companies have the highest salaries, followed by Germany, Netherlands, Austria and France. South European companies have the lowest salaries (which is consistent with their lower standard of living).

# Prediction

In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
df = pd.read_csv('Data/data_science_jobs_dataset.csv')

In [3]:
df.drop('salary_in_usd',axis=1,inplace=True)

In [4]:
df['work_year'] = df['work_year'].astype("string")
df['remote_ratio'] = df['remote_ratio'].astype("string")

In [5]:
df.drop('employment_type', axis=1, inplace=True)

In [6]:
job_title_cat = df.groupby('job_title')['job_title'].agg('count').sort_values(ascending=False)
job_title_cat_other = job_title_cat[job_title_cat<=3]
df.job_title = df.job_title.apply(lambda x: 'other' if x in job_title_cat_other else x )

In [7]:
emp_res_cat = df.groupby('employee_residence')['employee_residence'].agg('count').sort_values(ascending=False)
emp_res_cat_other = emp_res_cat[emp_res_cat<=2]
df.employee_residence = df.employee_residence.apply(lambda x: 'other' if x in emp_res_cat_other else x )

In [8]:
com_loc_cat = df.groupby('company_location')['company_location'].agg('count').sort_values(ascending=False)
com_loc_cat_other = com_loc_cat[com_loc_cat<=2]
df.company_location = df.company_location.apply(lambda x: 'other' if x in com_loc_cat_other else x )

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
train_data, test_data = train_test_split(df,test_size=0.2)

In [11]:
train = df.drop('salary',axis=1)
train_labels = df['salary']

In [12]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('one_hot_cat',OneHotEncoder())
])
 

train_pipelined = pipeline.fit_transform(train)

In [13]:
test = test_data.drop('salary',axis=1)
test_labels = test_data['salary']

In [14]:
test_pipelined = pipeline.transform(test)

In [15]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()

tree_reg.fit(train_pipelined,train_labels)

DecisionTreeRegressor()

In [16]:
tree_reg.score(test_pipelined,test_labels)

0.9998920893160212

In [17]:
df_pipelined = pipeline.transform(df.drop('salary', axis=1))

In [18]:
df["prediction"] = tree_reg.predict(df_pipelined)

In [44]:
df_pipelined

<811x110 sparse matrix of type '<class 'numpy.float64'>'
	with 6488 stored elements in Compressed Sparse Row format>

In [45]:
train_pipelined

<811x110 sparse matrix of type '<class 'numpy.float64'>'
	with 6488 stored elements in Compressed Sparse Row format>

In [46]:
test_pipelined

<163x110 sparse matrix of type '<class 'numpy.float64'>'
	with 1304 stored elements in Compressed Sparse Row format>

In [20]:
df.dtypes

work_year              string
experience_level       object
job_title              object
salary                  int64
salary_currency        object
employee_residence     object
remote_ratio           string
company_location       object
company_size           object
prediction            float64
dtype: object

In [26]:
pickle.dump(tree_reg, open("Predictive_model/tree_regressor.p","wb"))

In [23]:
### pickle.load(open("Predictive_model/tree_regressor.p","rb"))

In [50]:
pipeline

Pipeline(steps=[('one_hot_cat', OneHotEncoder())])

In [49]:
OneHotEncoder

sklearn.preprocessing._encoders.OneHotEncoder

In [51]:
pickle.dump(pipeline, open("Predictive_model/pipeline.p","wb"))